<a href="https://colab.research.google.com/github/EVA5-Stars/EVA5/blob/master/S6/EVA5_Submission_Session_6_Final(p).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Objective take best 5th code, and run below versions for 25 epochs and report findings:

>> 1. with L1 + BN
2. with L2 + BN
3. with L1 and L2 with BN
4. with GBN
5. with L1 and L2 with GBN


* *You need to write a single loop or iterator to iterate through these conditions.*

* Draw ONE graph to show the validation accuracy curves for all 5 jobs above. This graph must have proper legends and it should be clear what we are looking at.

* Draw ONE graph to show the loss change curves for all 5 jobs above. This graph must have proper legends and it should be clear what we are looking at.

* Find any 25 misclassified images (combined into single image) for "with GBN" model. You should be using the saved model from the above jobs.  You MUST show the actual and predicted class names.

Submitted By: 
* Gaurav Makkar
* Mohit Bhandari
* Suman Debnath
* Sriram Iyengar




## Import `pytorch` libraries and modules:

In [1]:
from __future__ import print_function
import torch
#import nueral net model in the torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#torchvision is the libraray in torch that has many features
#and we are using datasets and transforms from this library 
from torchvision import datasets, transforms

In [2]:
!rm -rf EVA5
!git clone https://github.com/EVA5-Stars/EVA5.git

Cloning into 'EVA5'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 138 (delta 74), reused 85 (delta 39), pack-reused 0
Receiving objects: 100% (138/138), 530.13 KiB | 775.00 KiB/s, done.
Resolving deltas: 100% (74/74), done.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls /content/gdrive/My\ Drive/Colab\ Notebooks/EVA5/S6

Mounted at /content/gdrive
EVA5_Submission_Session_6_inprogress.ipynb
EVA5:Submission-Session_6_Model-BatchNormalization.pt
EVA5:Submission-Session_6_Model-GhostBatchNormalization.pt


In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Project Library:

*   Ghost Batch Normalization
*   Misclassified image collection
*   Load the testing and training data with configurabe *batch_size*
*   Plot the Loss and Accuracy Graph
*   Define the structure of the network
*   Setup to create model and summary
*   Load the testing and training data with configurabe *batch_size*
*   Define test and train functions
*   Create a function to *initiate_training* for all conditions





**Ghost Batch Normalization**: 

A technique originally developed for training with very large batch sizes across many accelerators. 

In [6]:
#from EVA5.S6.BatchNorm_GhostBatchNorm import GhostBatchNorm
#from EVA5.S6.BatchNorm_GhostBatchNorm import BatchNorm

**Collecting Misclassified Images**

In [7]:
#from S6.S6_plot_misclassfied_image_finder import misclassified_image_finder              
def misclassified_image_finder(model, model_path, device, train_loader, image_num, msg):
    
    data_iter = iter(test_loader)
    figure = plt.figure()

    plt.title('Misclassified Images: With {}'.format(msg))
   
    for _i in range(image_num):
          data, target = data_iter.next()

          model.load_state_dict(torch.load(model_path)) 
          model.eval()

          data, target = data.to(device), target.to(device)

          output = model(data)
          pred = output.argmax(dim=1, keepdim=True) 

          for a in range(256):
              if(pred[a]!=target[a]):
                  
                  plt.subplot(5,5,_i+1)
                  plt.axis('off')
                  plt.imshow(data[a].cpu().numpy().squeeze(),cmap='gray_r')

Plot the Loss and Accuracy Graph

In [8]:
from EVA5.S6.S6_plot_stat_loss_acc import plot_stat

Define the structure of the network

In [9]:
from EVA5.S6.S6_Model_structure_create import Net

Setup to create model and summary

In [10]:
from EVA5.S6.S6_Model_structure_create import create_model

Load the testing and training data with configurabe *batch_size*

In [11]:
from EVA5.S6.S6_data_loader import init_train_test_loader

Define test and train functions

In [12]:
from tqdm import tqdm

# Loss train & test
train_losses = []
test_losses = []

# Accuracy train & test
train_acc = []
test_acc = []

# Total Loss & Accuracy
total_train_losses = []
total_train_accuracies = []

total_test_losses = []
total_test_accuracies = []


from EVA5.S6.S6_train_test_function import train
from EVA5.S6.S6_train_test_function import test   


Create a function to *initiate_training* for all conditions

In [13]:
from torch.optim.lr_scheduler import StepLR
from EVA5.S6.S6_init_training import init_training   


 Initialize train and test loader with Batch Size:128


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Run: Initialization and trainings

Create the model with normal BatchNormalization

In [15]:
model = create_model(device,gbatnor=False)
# Set the Epoch
EPOCHS = 25
model_name_bn = "EVA5:Submission-Session_6_Model-BatchNormalization.pt"
save_path_bn = F"/content/gdrive/My Drive/Colab Notebooks/EVA5/S6{model_name_bn}"
torch.save(model.state_dict(), save_path_bn)
print("Model Saved: {}".format(model_name_bn))

Create the instance of the Net class with GBN = False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 16, 24, 24]           1,152
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8            [-1, 8, 12, 12]             128
              ReLU-9            [-1, 8, 12, 12]               0
      BatchNorm2d-10            [-1, 8, 12, 12]              16
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]  

1. with L1 + BN


In [ ]:
train_loader, test_loader = init_train_test_loader()

# with L1 and BN
init_training(model, device, train_loader, EPOCHS,train_losses,train_acc,test_losses,test_acc, step_lr=False, l1_lambda=1e-5, l2_en=False)

total_train_losses.extend([train_losses])
total_train_accuracies.extend([train_acc])
total_test_losses.extend([test_losses])
total_test_accuracies.extend([test_acc])

  0%|          | 0/469 [00:00<?, ?it/s]


 Initialize train and test loader with Batch Size:128


Loss=0.23868992924690247 Batch_id=468 Accuracy=88.07: 100%|██████████| 469/469 [00:58<00:00,  7.99it/s]



 Epoch 0, lr 0.01


Loss=0.2391173094511032 Batch_id=0 Accuracy=95.31:   0%|          | 1/469 [00:00<00:57,  8.14it/s]

...

Loss=0.04353528469800949 Batch_id=468 Accuracy=98.07: 100%|██████████| 469/469 [00:58<00:00,  8.06it/s]



 Epoch 4, lr 0.01


Loss=0.10394685715436935 Batch_id=0 Accuracy=97.66:   0%|          | 1/469 [00:00<01:01,  7.63it/s]


Test set: Average loss: 0.0691, Accuracy: 9845/10000 (98.45%)



Loss=0.11252481490373611 Batch_id=356 Accuracy=98.23:  76%|███████▌  | 356/469 [00:44<00:13,  8.15it/s]

2. with L2 + BN


In [ ]:
# with L2 and BN

# Loss train & test
train_losses = []
test_losses = []
# Accuracy train & test
train_acc = []
test_acc = []
init_training(model, device, train_loader, EPOCHS,train_losses,train_acc,test_losses,test_acc, step_lr=False, l1_lambda=None, l2_en=True)
total_train_losses.extend([train_losses])
total_train_accuracies.extend([train_acc])
total_test_losses.extend([test_losses])
total_test_accuracies.extend([test_acc])

3. with L1 and L2 with BN


In [ ]:
# Loss train & test
train_losses = []
test_losses = []
# Accuracy train & test
train_acc = []
test_acc = []
# with L1, L2 and BN
init_training(model, device, train_loader, EPOCHS,train_losses,train_acc,test_losses,test_acc, step_lr= True, l1_lambda=1e-5,l2_en=True)
total_train_losses.extend([train_losses])
total_train_accuracies.extend([train_acc])
total_test_losses.extend([test_losses])
total_test_accuracies.extend([test_acc])

Create the model with Ghost BatchNormalization

In [ ]:
model = create_model(device, gbatnor=True)
model_name_gbn = "EVA5:Submission-Session_6_Model-GhostBatchNormalization.pt"
save_path_gbn = F"/content/gdrive/My Drive/Colab Notebooks/EVA5/S6{model_name_gbn}"
torch.save(model.state_dict(), save_path_gbn)
print("Model Saved: {}".format(model_name_gbn))

4. with GBN

In [ ]:
train_loader, test_loader = init_train_test_loader(256)
# Loss train & test
train_losses = []
test_losses = []
# Accuracy train & test
train_acc = []
test_acc = []
# with L1 and GBN
init_training(model, device, train_loader, EPOCHS,train_losses,train_acc,test_losses,test_acc, step_lr= False,l1_lambda= 1e-5,l2_en=False)
total_train_losses.extend([train_losses])
total_train_accuracies.extend([train_acc])
total_test_losses.extend([test_losses])
total_test_accuracies.extend([test_acc])

5. with L1 and L2 with GBN

In [ ]:
# with L1, L2 and GBN
# Loss train & test
train_losses = []
test_losses = []
# Accuracy train & test
train_acc = []
test_acc = []
init_training(model, device, train_loader, EPOCHS,train_losses,train_acc,test_losses,test_acc, step_lr= True, l1_lambda=1e-5,l2_en= True)
total_train_losses.extend([train_losses])
total_train_accuracies.extend([train_acc])
total_test_losses.extend([test_losses])
total_test_accuracies.extend([test_acc])

In [ ]:
# 
from EVA5.S6.S6_plot_stat_loss_acc import plot_stat
plot_stat(total_test_losses, 'Loss')

In [ ]:
# 
plot_stat(total_test_accuracies, 'Acc')

In [ ]:
misclassified_image_finder(model, save_path_gbn, device, train_loader, EPOCHS, "Ghost Batch Normalization")

## Summary:

A simple DNN with Total params=7,612 was created and trained for 25 epochs with L1, L2 Regularization, Batch Normalization and Ghost Batch Normalization. 

Following are the accuracy observed:

> 1. with L1 + BN => **99.29%**
2. with L2 + BN => **99.31%**
3. with L1 and L2 with BN => **99.47%**
4. with GBN => **98.9%**
5. with L1 and L2 with GBN => **99.33%**

Reference:
* GBN:  https://github.com/apple/ml-cifar-10-faster/blob/master/utils.py

